In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.1 MB/s eta 0:00:00


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 36.4 MB/s eta 0:00:00


In [ ]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4067771 sha256=e254aa43d5a50c1260653eb3a208ecee4427fece02964e4acdd54994a4d26e5b
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
Successfully built llama-cpp-python


In [ ]:
import os
import csv
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import faiss
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

# ----------------------
# Configuration
# ----------------------
PDF_DIR = "/content/prospectus"  # folder with all IPO PDFs
CHUNK_SIZE = 1000
QUESTIONS = [
    "How will the company utilize the funds raised through the IPO, and what are the stated objectives or purposes of the offering?",
    "What are the company’s key strengths and competitive advantages, how is it positioned among its competitors, and what is the industry outlook for future growth?",
    "What are the major internal and external risks that could impact the company’s performance or future prospects?"
]

# ----------------------
# Load LLM Model
# ----------------------
model_path = hf_hub_download(
    repo_id="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    filename="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    local_dir=".",
    local_dir_use_symlinks=False
)

llm = Llama(
    model_path=model_path,
    n_ctx=8192,
    n_threads=8,
    n_gpu_layers=0,
    use_mlock=False,
    use_mmap=True
)

# ----------------------
# Utilities
# ----------------------
def clean_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    text = text.replace('\n', ' ')
    return ' '.join(text.split())

def create_vector_store(text):
    chunks = [text[i:i + CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE)]
    embedder = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = embedder.encode(chunks, batch_size=32, show_progress_bar=False, convert_to_numpy=True)
    index = faiss.IndexFlatL2(384)
    index.add(embeddings)
    return chunks, embedder, index

def retrieve(query, chunks, embedder, index, top_k=8):
    q_embed = embedder.encode([query])
    _, I = index.search(q_embed, top_k)
    return "\n\n".join(chunks[i] for i in I[0])

def ask(query, chunks, embedder, index):
    context = retrieve(query, chunks, embedder, index)
    prompt = f"""
You are an IPO expert evaluating investment opportunities. Use only the context below to answer the question concisely and informatively.

Context:
{context}

Question:
{query}

Answer (respond with a structured paragraph):
"""
    response = llm(prompt, max_tokens=768, temperature=0.3, top_p=0.9, repeat_penalty=1.1)
    return response["choices"][0]["text"].strip()

# ----------------------
# Main Loop
# ----------------------
results = []
pdf_files = [f for f in os.listdir(PDF_DIR) if f.endswith(".pdf")]

for pdf_file in pdf_files:
    ipo_name = os.path.splitext(pdf_file)[0]
    print(f"Processing {ipo_name}...")

    try:
        full_text = clean_text_from_pdf(os.path.join(PDF_DIR, pdf_file))
        chunks, embedder, index = create_vector_store(full_text)

        answers = [ask(q, chunks, embedder, index) for q in QUESTIONS]
        results.append([ipo_name] + answers)

    except Exception as e:
        print(f"Error processing {ipo_name}: {e}")

# ----------------------
# Save to CSV
# ----------------------

csv_path = "/content/ipo_analysis_results.csv"
with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["IPO Name", "Fund Utilization & Objectives", "Key Strengths & Industry Outlook", "Risks"])
    writer.writerows(results)

print(f"\n Analysis complete. Results saved to {csv_path}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


mistral-7b-instruct-v0.1.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.atte

Processing abha_power_and_steel...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]